## <center>tensorflow基础<center>

Tensorflow的设计理念称之为计算流图，在编写程序时，首先构筑整个系统的graph，代码并不会直接生效，这一点和python的其他数值计算库（如Numpy等）不同，graph为静态的，类似于docker中的镜像。然后，在实际的运行时，启动一个session，程序才会真正的运行。这样做的好处就是：避免反复地切换底层程序实际运行的上下文，tensorflow帮你优化整个系统的代码。


tf.flags
用于帮助我们添加命令行的可选参数
FLAGS = tf.flags.FLAGS #FLAGS保存命令行参数的数据
FLAGS._parse_flags() #将其解析成字典存储到FLAGS.__flags中


tf.argmax(input, axis=None, name=None, dimension=None)
此函数是对矩阵按行或列计算最大值


- optimize_for_inference
通过调用 optimize_for_inference 脚本，会自动删除模型中输入层和输出层之间所有不需要的节点。
同时该脚本还做了一些其他优化以提高运行速度。例如它把显式批处理标准化运算跟卷积权重进行了合并，从而降低了计算量。

tflearn是TensorFlow的高级封装



In [3]:
import numpy as np
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

### tf.Session.run()

run(    fetches,   feed_dict=None,    options=None,    run_metadata=None)

tf.Session.run()函数返回值为fetches的执行结果。
如果fetches是一个元素就返回一个值；
若fetches是一个list，则返回list的值，
若fetches是一个字典类型，则返回和fetches同keys的字典。
feed_dict是一个字典，在字典中需要给出每一个用到的占位符的取值。


##### tf.session 配置
- log_device_placement=True : 是否打印设备分配日志
- allow_soft_placement=True ： 如果你指定的设备不存在，允许TF自动分配设备
- 使用allow_growth option，刚一开始分配少量的GPU容量，然后按需慢慢的增加，由于不会释放内存，所以会导致碎片

##### 控制使用哪块GPU
/ CUDA_VISIBLE_DEVICES=0  python your.py#使用GPU0
~/ CUDA_VISIBLE_DEVICES=0,1 python your.py#使用GPU0,1

- 注意单词不要打错
- 或者在 程序开头
- os.environ['CUDA_VISIBLE_DEVICES'] = '0' #使用 GPU 0
- os.environ['CUDA_VISIBLE_DEVICES'] = '0,1' # 使用 GPU 0，1

##### tf.app.run()

如果你的代码中的入口函数不叫main()，而是一个其他名字的函数，如test()，则你应该这样写入口tf.app.run(test)
如果你的代码中的入口函数叫main()，则你就可以把入口写成tf.app.run()



### tf.Graph()
tf.Graph() 默认执行
graph定义了计算方式，是一些加减乘除等运算的组合，类似于一个函数。它本身不会进行任何计算，也不保存任何中间计算结果。
session用来运行一个graph，或者运行graph的一部分。它类似于一个执行者，给graph灌入输入数据，得到输出，并保存中间的计算结果。
同时它也给graph分配计算资源（如内存、显卡等）。
op -- 操作
graph就是由一系列op构成的


In [4]:
x = tf.constant(-1)
y = tf.constant([-1 + 2j, -2])
sess = tf.Session()
# print(sess.run(tf.abs(x))) #1
# print(sess.run(tf.abs(y))) #[ 2.23606798，2.        ]

## 计算序列长度
used = tf.sign(tf.abs(y))
length = tf.reduce_sum(used)
# length = tf.cast(length, tf.int32)
print(sess.run([length,used]))

pooled_outputs = []
gmp1 = tf.Variable(tf.random_normal([2,10]))
gmp2 = tf.Variable(tf.random_normal([2,10]))
pooled_outputs.append(gmp1)
pooled_outputs.append(gmp2)
# Combine all the pooled features
h_pool = tf.concat(pooled_outputs, 1)
gmps = tf.reshape(h_pool, [-1, ])
print(gmps.shape)

[2.0, array([1., 1.])]
(40,)


### one-hot 

- tf.sequence_mask
- tf.boolean_mask

In [7]:
res = tf.cast(tf.sequence_mask([2,3]), tf.float32)

with tf.Session() as sess:
    # sess.run(tf.global_variables_initializer())
    output = sess.run(res)
    print("output of one-hot is : ", output)

output of one-hot is :  [[1. 1. 0.]
 [1. 1. 1.]]


In [6]:
res = tf.cast(tf.boolean_mask([2,3],mask=[1,3]),tf.float32)

with tf.Session() as sess:
    # sess.run(tf.global_variables_initializer())
    output = sess.run(res)
    print("output of one-hot is : ", output)

output of one-hot is :  [2. 3.]


#### tf读取数据
tf.data API能够从不同的输入或文件格式中读取、预处理数据，并且对数据应用一些变换
（例如，batching、shuffling、mapping function over the dataset），

tf.data API 是旧的 feeding、QueueRunner的升级。
基于C++的多线程及队列

Feeding是数据输入效率最低的方式，应该只用于小数据集和调试（debugging）
https://blog.csdn.net/u014061630/article/details/80712635


[TensorFlow高效读取数据的方法](https://blog.csdn.net/u012759136/article/details/52232266)

In [8]:
filename = './data/test.csv'
file_queue = tf.train.string_input_producer([filename])  # 设置文件名队列，这样做能够批量读取文件夹中的文件
reader = tf.TextLineReader(skip_header_lines=1)  # 使用tensorflow文本行阅读器，并且设置忽略第一行

key, value = reader.read(file_queue)

print(key,value)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TextLineDataset`.
Tensor("ReaderReadV2:0", shape=(), dt

In [9]:
sess = tf.Session()
data = tf.constant([[1, 2, 1], [3, 1, 1]])
# data = tf.Variable(tf.int32, [None, 10], name='input_x')
print(sess.run(tf.shape(data)))
d_1 = tf.expand_dims(data, 2)
# d_1 = tf.expand_dims(d_1, 2)
# d_1 = tf.expand_dims(d_1, -1)
# d_1 = tf.expand_dims(d_1, -1)
print(sess.run(tf.shape(d_1)))
d_2 = d_1
print(sess.run(tf.shape(tf.squeeze(d_1))))
# print(sess.run(tf.shape(tf.squeeze(d_2, [2, 4]))))

[2 3]
[2 3 1]
[2 3]


 #### tf.range函数用来快速生成一个等差数列 
    
https://blog.csdn.net/lusing/article/details/80054716

In [10]:
b11 = tf.range(1,100,1)  # 生成一个对象

a5 = tf.linspace(1.0,100.0, 10)  # linspace生成浮点等差数组 从1.0至100.0 10个元素

a6 = tf.slice(a5, [2], [4]) # 从index=2的元素开始，取4个元素

a20 = tf.linspace(1.0,2.0,10)
a21 = tf.linspace(2.0,3.0,5)
# 连接操作
a23 = tf.concat([a20,a21],-1)

# 广播操作： 向量和标量中间进行运行（加减）
b1 = tf.constant([1,2])
b2 = tf.constant([2,1])


# Hadamard积
b3 = b1 * b2   # b3 = b1 * b2   # element-wise product)

# 点积(dot product)
# a31 = tf.constant(tf.reshape(a21,[2,1]))
# a32 = tf.constant(tf.reshape(a21,[2,1]))
# a31 = tf.matmul(a31,a32,transpose_a=True)

with tf.Session() as sess:
    sess.run(b11)
    print(sess.run(a5))
    print(sess.run(a6))
    print(sess.run(a23))
    sess.run(a20)
    sess.run(a21)
    # print(sess.run(a23))
    print(sess.run(b1))  # [1 2]

    print(sess.run(b3))

    # tensordot要求是浮点型矩阵
    f01 = tf.constant([1, 1], dtype=tf.float32)
    f02 = tf.constant([1, 2], dtype=tf.float32)
    f11 = tf.constant(sess.run(tf.reshape(f01, [2, 1])))
    f12 = tf.constant(sess.run(tf.reshape(f02, [2, 1])))
    f13 = tf.tensordot(f11, f12, 2)
    print(sess.run(f13))

[  1.  12.  23.  34.  45.  56.  67.  78.  89. 100.]
[23. 34. 45. 56.]
[1.        1.1111112 1.2222222 1.3333334 1.4444444 1.5555556 1.6666667
 1.7777778 1.8888888 2.        2.        2.25      2.5       2.75
 3.       ]
[1 2]
[2 2]
3.0


In [12]:
x = tf.Variable(0)
xr = x.read_value()
y = x.assign(3)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(x))
    print(sess.run(xr))
    print(sess.run(y))
    
# 创建一个整型常量，即 0 阶 Tensor
t0 = tf.constant(3, dtype=tf.int32)
# 创建一个浮点数的一维数组，即 1 阶 Tensor
t1 = tf.constant([3., 4.1, 5.2], dtype=tf.float32)
# 创建一个字符串的2x2数组，即 2 阶 Tensor
t2 = tf.constant([['Apple', 'Orange'], ['Potato', 'Tomato']], dtype=tf.string)
# 创建一个 2x3x1 数组，即 3 阶张量，数据类型默认为整型
t3 = tf.constant([[[5], [6], [7]], [[4], [3], [2]]])

# 打印上面创建的几个 Tensor
print(t0)
print(t1)
print(t2)
print(t3)

0
0
3
Tensor("Const_13:0", shape=(), dtype=int32)
Tensor("Const_14:0", shape=(3,), dtype=float32)
Tensor("Const_15:0", shape=(2, 2), dtype=string)
Tensor("Const_16:0", shape=(2, 3, 1), dtype=int32)


#### tf.nn.embedding_lookup（tensor, id）函数的用法主要是选取一个张量里面索引(id)对应的元素

https://www.jianshu.com/p/abea0d9d2436

In [14]:
"""
lk[0]也就是[0,1]对应着下面sess.run(lookup_data)的结果恰好是把data中的[[2],[1]],[[3],[4]]
[
    [
        [[2][1]]
        [[3][4]]
    ]


    [
        [[3][4]]
        [[2][1]]
    ]


    [
        [[2][1]]
        [[2][1]]
    ]
]

"""
# 返回10*4的矩阵，产生于low和high之间，产生的值是均匀分布的。
w = tf.random_uniform([10, 4], -1.0, 1.0)
print(w)   # 10*4

print(tf.reduce_mean(w))

# tf.reduce_mean 可跨越维度的计算张量各元素的平均值
#
# embedded_chars = tf.nn.embedding_lookup(w, input_x)

data = np.array([[[2],[1]],[[3],[4]],[[6],[7]]])
data = tf.convert_to_tensor(data)
print(data.shape)  # 3*2*1
lk = tf.convert_to_tensor(np.array([[0,1],[1,0],[0,0]]))
print(lk.shape)
lookup_data = tf.nn.embedding_lookup(data,lk)
print(lookup_data.shape)

sess = tf.Session()
print(sess.run(lookup_data))

Tensor("random_uniform_1:0", shape=(10, 4), dtype=float32)
Tensor("Mean_1:0", shape=(), dtype=float32)
(3, 2, 1)
(3, 2)
(3, 2, 2, 1)
[[[[2]
   [1]]

  [[3]
   [4]]]


 [[[3]
   [4]]

  [[2]
   [1]]]


 [[[2]
   [1]]

  [[2]
   [1]]]]


In [19]:
T = tf.random_uniform([10, 1], -1.0, 1.0)
C = tf.subtract(1.0, T, name="carry_gate")  #　Returns x - y element-wise.
C

<tf.Tensor 'carry_gate_1:0' shape=(10, 1) dtype=float32>

In [20]:
T

<tf.Tensor 'random_uniform_3:0' shape=(10, 1) dtype=float32>

- tf.random_normal    从正态分布中输出随机值。
- tf.truncated_normal  从截断的正态分布中输出随机值;生成的值服从具有指定平均值和标准偏差的正态分布
- tf.random_uniform 从均匀分布输出随机值。生成的值遵循该范围内的均匀分布 [minval, maxval)。下限minval包含在范围内，而maxval排除上限

In [7]:
seq_length = 4   #  3*4 = 12
embedding_dim = 5

input = tf.Variable(tf.random_normal([10,seq_length,embedding_dim]))
input

<tf.Variable 'Variable:0' shape=(10, 4, 5) dtype=float32_ref>

In [8]:
input_att = tf.split(input, seq_length, axis=1) # 提取张量的部分
input_att

[<tf.Tensor 'split:0' shape=(10, 1, 5) dtype=float32>,
 <tf.Tensor 'split:1' shape=(10, 1, 5) dtype=float32>,
 <tf.Tensor 'split:2' shape=(10, 1, 5) dtype=float32>,
 <tf.Tensor 'split:3' shape=(10, 1, 5) dtype=float32>]

In [10]:
outputs = []
conttexts = []  # 10,4,3,5
for index, x_index in enumerate(input_att):
    tmp1 = []
    tmp2 = []
    # x_i = tf.reshape(x_index, [-1, embedding_dim])
    # print(x_i)  # Tensor("split:0", shape=(10, 1, 4), dtype=float32)  Tensor("Reshape:0", shape=(10, 4), dtype=float32)
    for indxe, x_indxe in enumerate(input_att):
        # print(indxe)
        if index != indxe:
            # print(x_indxe.shape)
            tmp1.append(tf.concat([x_index,x_indxe], axis=2))
            tmp2.append(x_indxe)
    tmp1 = tf.concat(tmp1, axis=1)
    tmp2 = tf.concat(tmp2, axis=1)
    # print(tmp1.shape)
    outputs.append(tf.expand_dims(tmp1,1))
    conttexts.append(tf.expand_dims(tmp2, 1))
outputs = tf.concat(outputs, axis=1)
conttexts = tf.concat(conttexts, axis=1)
conttexts

<tf.Tensor 'concat_43:0' shape=(10, 4, 3, 5) dtype=float32>

In [13]:
hidden_dim = 3
W = tf.Variable(tf.random_normal([embedding_dim*2, hidden_dim]))
V = tf.Variable(tf.random_normal([hidden_dim,1]))
outputs = tf.reshape(outputs,[-1,embedding_dim*2])
WX = tf.matmul(outputs, W)
# print(WX.shape) # (120, 3)

VWX = tf.matmul(WX, V)
# print(VWX.shape)
VWX = tf.reshape(VWX,[-1,3])
# print(VWX.shape)
VWX = tf.exp(VWX)
# print(VWX.shape)
alpha_M = tf.reshape(tf.nn.softmax(VWX,1),[10,4,3])
alpha_M

<tf.Tensor 'Reshape_5:0' shape=(10, 4, 3) dtype=float32>

### tf.tile

tensorflow中的tile()函数是用来对张量(Tensor)进行扩展的，其特点是对当前张量内的数据进行一定规则的复制。

In [14]:
alpha_MM = tf.tile(tf.expand_dims(alpha_M,3),[1,1,1,embedding_dim])
alpha_MM

<tf.Tensor 'Tile_1:0' shape=(10, 4, 3, 5) dtype=float32>

#### tf.abs()就是求数值的绝对值，可以将多个数值传入list，统一求绝对值

In [29]:
x = tf.constant(-1)
y = tf.constant([-1 + 2j, -2])
sess = tf.Session()
print(sess.run(tf.abs(x))) #1
print(sess.run(tf.abs(y))) #[ 2.23606798，2.        ]

1
[2.23606798 2.        ]


### tf.gather 
用一个一维的索引数组，将张量中对应索引的向量提取出来

In [22]:
a = tf.Variable([[1,2,3,4,5], [6,7,8,9,10], [11,12,13,14,15]])
index_a = tf.Variable([0,2])
 
b = tf.Variable([1,2,3,4,5,6,7,8,9,10])
index_b = tf.Variable([2,4,6,8])
 
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(tf.gather(a, index_a)))
    print(sess.run(tf.gather(b, index_b)))

[[ 1  2  3  4  5]
 [11 12 13 14 15]]
[3 5 7 9]
